In [1]:
import numpy as np
import pandas as pd 
import pandas_profiling as pp
import plotly.express as px 
import sqlite3
import random
from sklearn.metrics import mean_squared_error

pd.options.display.max_columns = None

## Import des datasets 

In [2]:
df_aeroports = pd.read_parquet("../data/aggregated_data/aeroports.gzip")
df_compagnies = pd.read_parquet("../data/aggregated_data/compagnies.gzip")
df_vols = pd.read_parquet("../data/aggregated_data/vols.gzip")
df_fuel = pd.read_parquet("../data/aggregated_data/prix_fuel.gzip")
df_test = pd.read_parquet("../data/extracted/test_data/vols.gzip")

In [18]:
df_compagnies

,COMPAGNIE,CODE,NOMBRE D EMPLOYES,CHIFFRE D AFFAIRE
0,Try Hard Airlines,THA,12909,2310000000
1,Corporate Overlord Airways,COA,130595,40579000000
2,Neverland Airlines,NA,18567,7651000000
3,Always A Problem Flights,AAPF,92000,51000000000
4,Overpriced Air,OA,12200,6649000000
5,Morally Ambiguious Fligthts,MAF,15150,6391000000
6,Fliying Is Possible Inc.,FIPI,13230,2979000000
7,Air Piloter Sans Les Mains,APSLM,13112,7651000000
8,Ne Va Pas Partout Airlines,NVPPA,12000,6235000000
9,IE 6.0 Flights,I6F,17569,2660000000


In [4]:
df_vols['COMPAGNIE AERIENNE']

0            MAF
1            I6F
2          NVPPA
3          NVPPA
4            THA
           ...  
4332909    SMITH
4332910    SMITH
4332911    SMITH
4332912    SMITH
4332913    SMITH
Name: COMPAGNIE AERIENNE, Length: 4332914, dtype: object

In [5]:
list_compagnies = df_compagnies['COMPAGNIE'].unique().tolist()
list_airports = df_aeroports['CODE IATA'].unique().tolist()

In [6]:
airlines = [random.choice(list_compagnies) for i in range(100)]
airports = [random.choice(list_airports) for i in range(100)]
nb_passagers = list(random.sample(range(0, 2505), 100))

In [7]:
y_true = np.array(random.sample(range(-100, 1898), 100))
y_preds = np.array(random.sample(range(-100, 1898), 100))
prediction = pd.DataFrame({'RETARD': y_preds, 'COMPAGNIES': airlines, 'AEROPORTS': airports,
                          'NOMBRE DE PASSAGERS': nb_passagers, 'RETARD REEL' : y_true})

# METRICS 

On a choisit la **RMSE** : La racine de l'erreur quadratique moyenne ou racine de l'écart quadratique moyen est une mesure fréquemment utilisée des différences entre les valeurs prédites par un modèle ou estimateur et les valeurs observées

- **Formule** : $RMSE = \sqrt{MSE(\theta)} = \sqrt{\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)^{2}}$

- **Interpretation** : plus la valeur de notre RMSE est grande plus notre prédiction (le retard prédit à l'arrivée) est loin de la réalité terrain, du retard à l'arrivée effectif. Une RMSE proche de 0 signifie que notre modèle est proche de la réalité dans ces prédictions. 

On a également choisi une deuxième métrique : 
**Moyenne** : $\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)$

**Objectif** : pénaliser l'écart entre la prédiction et la réalité terrain mais aussi pénaliser la direction de l'erreur, c'est à dire si notre modèle prédit plus de retard que prévu ou à l'inverse moins de retard que prévu. 

In [8]:
airline_list = list(dict.fromkeys(airlines))
rmse = []
mean_error=[]
for idx, airline in enumerate(airline_list): 
    y_true = prediction[prediction['COMPAGNIES'] == airline]['RETARD REEL']
    y_preds =  prediction[prediction['COMPAGNIES'] == airline]['RETARD']
    rmse.append(mean_squared_error(y_true, y_preds, squared=False)) #if squared=True return MSE value
    mean_error.append((y_true - y_preds).mean())

df_metrics = pd.DataFrame({'COMPAGNIE':airline_list, 'RMSE': rmse, 'MEAN ERROR': mean_error})

In [9]:
df_metrics

,COMPAGNIE,RMSE,MEAN ERROR
0,Corporate Overlord Airways,990.362896,-229.333333
1,Air Penguin,977.158015,75.642857
2,Always A Problem Flights,1110.744720,-296.500000
3,Air Piloter Sans Les Mains,913.839762,-455.555556
4,Morally Ambiguious Fligthts,570.653719,214.666667
5,Overpriced Air,397.679519,178.384615
6,Ne Va Pas Partout Airlines,849.791739,235.000000
7,IE 6.0 Flights,1245.031124,-1076.500000
8,Bel Air,1033.200852,248.666667
9,Fliying Is Possible Inc.,240.625851,111.200000


# KPIs

In [10]:
prediction["CHIFFRE D'AFFAIRE COMPAGNIE"] = prediction['COMPAGNIES'].map(lambda x:\
                                       df_compagnies[df_compagnies['COMPAGNIE'] ==x]['CHIFFRE D AFFAIRE'].values[0])

In [11]:
prediction_avec_retard = prediction[prediction['RETARD']>0].copy()

### Prix du retard

Hypothese prix retard aéroport (centaine d'euros) : 
- après 10min :  la compagnie paye toutes les minutes le prix indiqué dans la colonne "PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES"
- après 20min : la compagnie paye un **supplément** qui est le prix indiqué dans la colonne "PRIX RETARD PREMIERE 20 MINUTES"

In [13]:
print(len(prediction_avec_retard))

96


In [14]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE
0,146,Corporate Overlord Airways,CKG,939,391,40579000000
1,799,Air Penguin,CPH,2326,220,31064000000
2,731,Air Penguin,ICN,1029,-37,31064000000
3,264,Always A Problem Flights,DAM,2484,54,51000000000
4,68,Air Piloter Sans Les Mains,OVB,171,308,7651000000
...,...,...,...,...,...,...
95,895,Overpriced Air,PEK,885,1007,6649000000
96,511,Air Penguin,TPE,134,1854,31064000000
97,430,Morally Ambiguious Fligthts,HAN,472,1409,6391000000
98,426,Ne Va Pas Partout Airlines,NAP,1375,934,6235000000


In [16]:
test = pd.merge(prediction_avec_retard, 
                df_aeroports[['CODE IATA', 'PRIX RETARD PREMIERE 20 MINUTES']].rename(columns={'CODE IATA': 'AEROPORTS'}),
                on='AEROPORTS', how='left')
test 

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE,PRIX RETARD PREMIERE 20 MINUTES
0,146,Corporate Overlord Airways,CKG,939,391,40579000000,67
1,799,Air Penguin,CPH,2326,220,31064000000,58
2,731,Air Penguin,ICN,1029,-37,31064000000,27
3,264,Always A Problem Flights,DAM,2484,54,51000000000,41
4,68,Air Piloter Sans Les Mains,OVB,171,308,7651000000,42
...,...,...,...,...,...,...,...
91,895,Overpriced Air,PEK,885,1007,6649000000,45
92,511,Air Penguin,TPE,134,1854,31064000000,21
93,430,Morally Ambiguious Fligthts,HAN,472,1409,6391000000,83
94,426,Ne Va Pas Partout Airlines,NAP,1375,934,6235000000,90


In [17]:
print(len(test))

96


In [ ]:
def add_cost_20min_delay(df_aeroports, airport):
    twenty_first_min_cost = df_aeroports[
        df_aeroports['CODE IATA'] == airport]['PRIX RETARD PREMIERE 20 MINUTES'].values[0]
    return twenty_first_min_cost

def add_cost_10min_delay(df_aeroports, airport):
    ten_min_delay_cost = df_aeroports[
            df_aeroports['CODE IATA'] == airport]['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'].values[0]
    return ten_min_delay_cost

In [ ]:
def cost_of_delay(pred_vol):
    delay = pred_vol['RETARD']
    twenty_first_min_cost = pred_vol['PRIX RETARD PREMIERE 20 MINUTES']
    ten_min_delay_cost = pred_vol['PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES']
    
    cost = 0
    if delay > 10 : 
        cost += ten_min_delay_cost * (delay - 10) 
    if delay >= 20 : 
        cost += twenty_first_min_cost
    return cost

In [ ]:
prediction_avec_retard['PRIX RETARD PREMIERE 20 MINUTES'] = prediction_avec_retard['AEROPORTS']\
                                                            .map(lambda x: add_cost_20min_delay(df_aeroports, x))
    
prediction_avec_retard['PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES'] = prediction_avec_retard['AEROPORTS']\
                                                                .map(lambda x: add_cost_10min_delay(df_aeroports, x))
prediction_avec_retard['COUT DU RETARD'] = prediction_avec_retard.apply(cost_of_delay, axis=1)

In [ ]:
prediction_avec_retard = prediction_avec_retard.drop(
    columns=['PRIX RETARD PREMIERE 20 MINUTES', 'PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES'])

### Indemnisation des clients 

Hypothèse : 
- 10% des clients vont demander à être indemnisé pour un retard compris entre 10min et 45min
    - Indemnité à payer : 1/4 du prix du billet
- 20% des clients vont demander à être indemnisé pour un retard supérieur à 1h 
    - Indemnité à payer : 1/2 du prix du billet
- 50% des clients vont demander à être indemnisé pour un retard supérieur à 3h 
    - Indemnité à payer : totalité du prix du billet
    
On fait l'hypothèse d'un fixe maximal du prix du billet : **300€**

In [ ]:
def get_number_of_indemnities_asked(pred_vol): 
    delay = pred_vol.loc['RETARD']
    nb_of_passenger = pred_vol.loc['NOMBRE DE PASSAGERS']
    nb_of_indemnities_asked = 0
    if delay > 10 and delay <45: 
        nb_of_indemnities_asked = 20*nb_of_passenger//100
    elif delay > 60 and delay <180:
        nb_of_indemnities_asked = 50*nb_of_passenger//100
    elif delay > 180:
        nb_of_indemnities_asked = 75*nb_of_passenger//100
    return nb_of_indemnities_asked

def compensation_due(pred_vol, ticket_price=300): 
    delay = pred_vol.loc['RETARD']
    nb_of_indemnities_asked = pred_vol.loc["NOMBRE D'INDEMNITES DEMANDEES"]
    compensation_due_to_clients = 0
    if delay > 10 and delay <45: 
        compensation_due_to_clients = (ticket_price/3)*nb_of_indemnities_asked
    elif delay > 60 and delay <180:
        compensation_due_to_clients = (ticket_price/2)*nb_of_indemnities_asked
    elif delay > 180:
        compensation_due_to_clients = ticket_price*nb_of_indemnities_asked
    return compensation_due_to_clients

In [ ]:
prediction_avec_retard[
    "NOMBRE D'INDEMNITES DEMANDEES"] = prediction_avec_retard.apply(get_number_of_indemnities_asked, axis=1)
prediction_avec_retard[
    "INDEMNITES A PAYER"] = prediction_avec_retard.apply(compensation_due, axis=1)

## Perte de client : 

**Hypothèse** : Taux d'attrition à 3% pour un retard de plus de 3h 

In [ ]:
def get_number_of_lost_customer(delay, passenger_nb):
    if delay > 180 : 
        return passenger_nb*3//100
    else : return 0

prediction_avec_retard['NOMBRE DE CLIENTS PERDUS'] = prediction_avec_retard.apply(
    lambda x: get_number_of_lost_customer(x["RETARD"], x['NOMBRE DE PASSAGERS']), axis=1)

In [ ]:
prediction_avec_retard

### Get the cost of all the lost client for the airlines

**Hypothèse** : 

On suppose qu'un client prend en moyenne 3 fois l'avion par an avec la même compagnie (on suppose une fidéité total des clients auprès de leur compagnie).

Donc si la compagnie perd un client, elle perd un cout de **3x"prix du billet"** par client

On suppose le prix du billet = 300€

In [ ]:
def get_cost_of_lost_customer(nb_of_lost_customers, ticket_price=300, flight_frequency=3):
    return flight_frequency*ticket_price*nb_of_lost_customers

In [ ]:
prediction_avec_retard['COUT DES CLIENTS PERDUS'] = prediction_avec_retard["NOMBRE DE CLIENTS PERDUS"].map(
                                                                        lambda x: get_cost_of_lost_customer(x))

In [ ]:
prediction_avec_retard

# TODO : 

- [x] Rajouter colonne avec pourcentage du chiffre d'affaire perdu 
- [x] Afficher nombre de vols en retard par compagnie 
- [x] combien de clients ils vont perdre : plus de 3h de retard --> taux d'attrition de 5% des clients
- [x] fréquence moyenne de réservation de vol avec la compagnie (3fs/an) : 5%x3x(prix du billet)
- [x] afficher le détail répartition des couts dans total à payer dans les graphes
- [ ] NEXT STEPS 

# BILAN : TOTAL A PAYER

In [ ]:
cost_of_delay_gb_airlines = prediction_avec_retard[["RETARD", "COMPAGNIES","CHIFFRE D'AFFAIRE COMPAGNIE",
                                                    "COUT DU RETARD", "INDEMNITES A PAYER", "NOMBRE DE CLIENTS PERDUS", 
                                                    "COUT DES CLIENTS PERDUS"]]\
                            .groupby(['COMPAGNIES'], as_index=False)\
                            .agg({
                                "RETARD" : "count",
                                "CHIFFRE D'AFFAIRE COMPAGNIE":'first',
                                "COUT DU RETARD":'sum',
                                "INDEMNITES A PAYER":'sum',
                                "NOMBRE DE CLIENTS PERDUS": "sum", 
                                "COUT DES CLIENTS PERDUS": "sum"
                            }).rename(columns={"RETARD" : "NOMBRE DE RETARD"})

In [ ]:
cost_of_delay_gb_airlines["TOTAL A PAYER"] = cost_of_delay_gb_airlines["COUT DU RETARD"]\
                                            + cost_of_delay_gb_airlines["INDEMNITES A PAYER"]\
                                            + cost_of_delay_gb_airlines["COUT DES CLIENTS PERDUS"]

In [ ]:
cost_of_delay_gb_airlines["NV CHIFFRE D'AFFAIRE"] = cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"]\
                                                    - cost_of_delay_gb_airlines["TOTAL A PAYER"]

In [ ]:
cost_of_delay_gb_airlines["%CHIFFRE D'AFFAIRE LOST"] = \
(cost_of_delay_gb_airlines["TOTAL A PAYER"]/cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"])*100

In [ ]:
cost_of_delay_gb_airlines

In [ ]:
fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "NV CHIFFRE D'AFFAIRE"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "TOTAL A PAYER"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

In [ ]:
import plotly.graph_objects as go

for idx, company in enumerate(cost_of_delay_gb_airlines["COMPAGNIES"]):
    labels = ["NV CHIFFRE D'AFFAIRE","TOTAL A PAYER"]
    values = [cost_of_delay_gb_airlines.iloc[idx]["NV CHIFFRE D'AFFAIRE"],
              cost_of_delay_gb_airlines.iloc[idx]["TOTAL A PAYER"]]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
    fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(title_text=company)
    fig.show()